Experiment settings

In [1]:
import simpy
import numpy as np
import pandas as pd
import random
import math
RANDOM_SEED = 42
### problem paramters ####
order_time_max = 3
order_time_min = 1
food_prepare_min = 2
food_prepare_max = 4


lamb = 0.4
n_kitchen = 3
n_counter = 2
n_customer = 200
SIM_TIME = 12 * 60 #12 hours

random.seed(RANDOM_SEED)

# helper functions, monitoring and pandas table


In [2]:
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_service_times = 0
counter_total_idle_times = 0
env = simpy.Environment()
# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Service Start Time","Queue time","Food prepare start","Exit system time","Total Wait Time (Queue+Food)","Service Time","Total Time in System"]
result_fifo = np.zeros((n_customer,len(column_names)))
result_tick = np.zeros((n_customer,len(column_names)))


# Simulation setting
## A) Ticket Algorithim

In [3]:
class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        self.counter_waiting = 0
        
    def take_order(self,cus,env, service_start):
        global counter_total_service_times

        print("%s is placing order at counter %.2f" %(cus,service_start))
        time_taken_to_place_order = np.random.uniform(order_time_min, order_time_max)
        yield self.env.timeout(time_taken_to_place_order)
        service_end = env.now
        print("Order of %s sent to kitchen at %.2f" %(cus, service_end))
        counter_total_service_times += (service_end-service_start)

    def receive_order(self,cus,env,kitchen):
        
        with kitchen.kitchen.request() as my_turn:
            yield my_turn
            yield env.process(kitchen.prepare_food(cus,env))
            food_end = env.now
            print("%s collected the food at %.2f" %(cus, food_end))
            # Record idle counter and add to total count
           

class Kitchen(object):
    # Kitchen to prepare food
    def __init__(self,env,num_kitchen):
        self.env = env
        self.kitchen = simpy.Resource(env,num_kitchen)
    
    def prepare_food(self,cus,env):
        print("Kitchen is preparing food for %s at %.2f" %(cus, env.now))
        food_prepare_time = np.random.uniform(food_prepare_min,food_prepare_max)
        yield self.env.timeout(food_prepare_time)
        print("Cooked food for %s at %.2f" %(cus, env.now))

def customer(env, label, queue, kitchen, data):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with queue.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,1] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,2]= round_down(queue_time)
        # placing order at counter
        yield env.process(queue.take_order(label,env,service_start))
        # waiting order at counter
        prepare_food_start = env.now
        data[label,3] = round_down(prepare_food_start)
        yield env.timeout(1)
    yield env.process(queue.receive_order(label,env,kitchen))
    # prepare_food_end = round_down(env.now)
    # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
    # receive food from counter
    exit_time = env.now
    data[label,4] = round_down(exit_time)

    kitchen_prepare_duration = (exit_time-data[label,3])
    # total wait time
    data[label,5] = round_down(kitchen_prepare_duration+data[label,2])
    # total service timeLIO
    data[label,6] = round_down(exit_time-service_start)
    # total time in system
    data[label,7] = round_down(exit_time-arrive_time)


## Start Ticket Simulation

In [4]:
# Simlating possion process for customer arrival
def customer_arrivals(env, num_counter,num_kitchen):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    counter = Counter(env,num_counter)
    kitchen = Kitchen(env,num_kitchen)
    for i in range(n_customer):
        yield env.timeout(random.expovariate(lamb))
        env.process(customer(env, i+1, counter, kitchen, result_fifo))

env.process(customer_arrivals(env, n_counter,n_kitchen))
env.run(until=SIM_TIME)

0 entering the queue at 2.55
0 is placing order at counter 2.55
1 entering the queue at 2.61
1 is placing order at counter 2.61
2 entering the queue at 3.42
Order of 0 sent to kitchen at 3.71
3 entering the queue at 4.05
Order of 1 sent to kitchen at 4.38
Kitchen is preparing food for 0 at 4.71
2 is placing order at counter 4.71
Kitchen is preparing food for 1 at 5.38
3 is placing order at counter 5.38
Order of 2 sent to kitchen at 7.22
4 entering the queue at 7.38
Cooked food for 1 at 7.52
1 collected the food at 7.52
Order of 3 sent to kitchen at 7.81
Cooked food for 0 at 7.82
0 collected the food at 7.82
Kitchen is preparing food for 2 at 8.22
4 is placing order at counter 8.22
Kitchen is preparing food for 3 at 8.81
Order of 4 sent to kitchen at 9.84
5 entering the queue at 10.21
5 is placing order at counter 10.21
Cooked food for 2 at 10.55
2 collected the food at 10.55
Kitchen is preparing food for 4 at 10.84
Cooked food for 3 at 10.95
3 collected the food at 10.95
Order of 5 sen

## ticket Result & Analysis

In [5]:
# Converting to pandas
np_arr = np.array(result_fifo).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)
# df_fifo
df_fifo=df_fifo.loc[~(df_fifo==0).all(axis=1)]
total_wait_time = df_fifo.iloc[:,5].sum()
total_service_time = df_fifo.iloc[:,6].sum()

total_time_in_system = df_fifo.iloc[:,7].sum()
print("")

In [6]:
df_fifo.iloc[1:30,:]

,Arrival Time,Service Start Time,Queue time,Food prepare start,Exit system time,Total Wait Time (Queue+Food),Service Time,Total Time in System
2,2.6,2.6,0.0,4.4,7.5,3.1,4.9,4.9
3,3.4,4.7,1.3,7.2,10.6,4.7,5.8,7.1
4,4.0,5.4,1.3,7.8,10.9,4.4,5.6,6.9
5,7.4,8.2,0.8,9.8,13.1,4.1,4.9,5.7
6,10.2,10.2,0.0,12.8,16.8,4.0,6.6,6.6
7,15.8,15.8,0.0,18.3,23.2,4.9,7.5,7.5
8,16.0,16.0,0.0,18.6,22.4,3.8,6.4,6.4
9,17.4,19.3,1.9,20.3,25.3,6.9,6.0,7.9
10,17.4,19.6,2.2,21.4,25.3,6.1,5.7,7.8
11,18.1,21.3,3.3,23.8,27.2,6.7,5.8,9.1


In [7]:
n_served = df_fifo.shape[0]

def display(i):
    print()
    print("%d Customers served"%n_served)
    print("Total Simulation Time=> %.2f Minutes" % SIM_TIME)
    print("Counter Total Idle Time => %.2f Minutes" % (SIM_TIME - counter_total_service_times))
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_served)


200 Customers served
Total Simulation Time=> 720.00 Minutes
Counter Total Idle Time => 321.59 Minutes

Average Waiting Time => 5.09 Minutes
Average Service Time => 5.97 Minutes
Average Time Spent In System => 7.08 Minutes
